# PVICE & RELOG - MassFlows Calculations

## 1. Initial setup

### Load libraries

In [1]:
import PV_ICE
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt

### Select the baselines and simulation folders

In [2]:
cwd = os.getcwd()
baselinefolder = os.path.join(cwd, 'baselines')
testfolder = os.path.join(cwd, 'TEMP')


In [3]:
SFscenarios = ['95-by-35_Elec.Adv_DR_cSi', '95-by-35_Elec.Adv_DR_CdTe']

### Reading GIS inputs

In [ ]:
from geopy.geocoders import Nominatim
from geopy.point import Point
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

In [ ]:
GISfile = os.path.join(baselinefolder, 'gis_centroid_n.xlsx')
GIS = pd.read_excel(GISfile)
GIS = GIS.set_index('id')

## 2. Load PCA baselines, create the 2 Scenarios and assign baselines

Keeping track of each scenario as its own PV ICE Object.

Select the method folder you want to run (uncomment your choice). There are three choices:
1. Method 1: Uses the raw regionalized capacity by ReEEDS, this creates a very uneven peak of wastes.
2. Method 2: Uses ordered wastes between 2021 to 2035 and 2046 to 2050. Still creates unrealistic peaks.
3. Method 3: Uses the cummulative capacity between 2021 to 2035 and 2034 to 2050 to create a logarithmic growth of waste (this method is being tested, not validated yet, and subjected to ongoing changes).

In [5]:
pv_ice_simulations = 'fsdf'

In [ ]:
method_list = ['Method1', 'Method2', 'Method3']

In [ ]:
pv_ice_simulations in method_list

In [7]:
print("Choose one of the following methods: Method1, Method2 or Method3."
pv_ice_simulations = None
while pv_ice_simulations not in {"Method1", "Method2", "Method3"}:
    pv_ice_simulations = input('Input method not in the list, please choose one of the following methods: Method1, Method2 or Method3.')

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (1904178949.py, line 1)

In [6]:
method_list = ['Method1', 'Method2', 'Method3']
while True:
    try:
        pv_ice_simulations = input('Choose one of the following methods: Method1, Method2 or Method3.')
    except ValueError:
        print('Input method not in the list, please choose one of the following methods: Method1, Method2 or Method3.')
        continue
    else:
        print('You have chosen {}.'.format(pv_ice_simulations))
        break

In [ ]:
method_list = ['Method1', 'Method2', 'Method3']
pv_ice_simulations = input('Choose one of the following methods: Method1, Method2 or Method3.')
if pv_ice_simulations not in method_list:
    while False:
        pv_ice_simulations = input('Input method not in the list, please choose one of the following methods: Method1, Method2 or Method3.')
print('You have chosen {}.'.format(pv_ice_simulations))

In [ ]:
pv_ice_simulations

### Scenario creation

In [ ]:
reedsFile = os.path.join(cwd, 'baselines','December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx')
print ("Input file is stored in %s" % reedsFile)
REEDSInput = pd.read_excel(reedsFile, sheet_name="new installs PV")
rawdf = REEDSInput.copy()
rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True) #tech=pvtotal from "new installs PV sheet", so can drop
rawdf.set_index(['Scenario','Year','PCA'], inplace=True)
PCAs = list(rawdf.unstack(level=2).iloc[0].unstack(level=0).index.unique())

In [ ]:
#for ii in range (0, 1): #len(scenarios):
i = 0
r1 = PV_ICE.Simulation(name=SFscenarios[i], path=testfolder)

for jj in range (0, len(PCAs)): 
    filetitle = SFscenarios[i]+'_'+PCAs[jj]+'.csv'
    filetitle = os.path.join(testfolder, f'PCAs_RELOG_{pv_ice_simulations}', filetitle)    # Change this number to the simulation you want to run
    r1.createScenario(name=PCAs[jj], massmodulefile=filetitle)
    r1.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=baselinefolder)
    # All -- but these where not included in the Reeds initial study as we didnt have encapsulant or backsheet
    # r1.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=r'..\baselines')
    r1.scenario[PCAs[jj]].latitude = GIS.loc[PCAs[jj]].lat
    r1.scenario[PCAs[jj]].longitude = GIS.loc[PCAs[jj]].long

r1.trim_Years(startYear=2010, endYear=2050)

i = 1
r2 = PV_ICE.Simulation(name=SFscenarios[i], path=testfolder)

for jj in range (0, len(PCAs)): 
    filetitle = SFscenarios[i]+'_'+PCAs[jj]+'.csv'
    filetitle = os.path.join(testfolder, f'PCAs_RELOG_{pv_ice_simulations}', filetitle)        
    r2.createScenario(name=PCAs[jj], massmodulefile=filetitle)
    # MAC Add here the materials you want.
    r2.scenario[PCAs[jj]].addMaterials(['cadmium', 'tellurium', 'glass_cdte', 'aluminium_frames_cdte', 'encapsulant_cdte', 'copper_cdte'], baselinefolder=baselinefolder)
    # All -- but these where not included in the Reeds initial study as we didnt have encapsulant or backsheet
    # r2.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=r'..\baselines')
    r2.scenario[PCAs[jj]].latitude = GIS.loc[PCAs[jj]].lat
    r2.scenario[PCAs[jj]].longitude = GIS.loc[PCAs[jj]].long

r2.trim_Years(startYear=2010, endYear=2050)


### Set characteristics for Manufacturing (probably don't want to inflate this as the waste happens elsewhere, just want EOL


In [ ]:
PERFECTMFG = True
# Set to false if I want to see how much goes to mnf waste
if PERFECTMFG:
    r1.scenMod_PerfectManufacturing()
    r2.scenMod_PerfectManufacturing()
    title_Method = 'PVICE_PerfectMFG'
else:
    title_Method = 'PVICE'

## 3. Calculate Mass Flow

In [ ]:
r1.calculateMassFlow()
r2.calculateMassFlow()

In [ ]:
print("PCAs:", r1.scenario.keys())
print("Module Keys:", r1.scenario[PCAs[jj]].dataIn_m.keys())
print("Material Keys: ", r1.scenario[PCAs[jj]].material['glass'].matdataIn_m.keys())

In [ ]:
"""
r1.plotScenariosComparison(keyword='Cumulative_Area_disposedby_Failure')
r1.plotMaterialComparisonAcrossScenarios(material='silicon', keyword='mat_Total_Landfilled')
r1.scenario['p1'].dataIn_m.head(21)
r2.scenario['p1'].dataIn_m.head(21)
r3.scenario['p1'].dataIn_m.head(21)
"""
pass

## 4. Aggregate & Save Data

In [ ]:
r1.aggregateResults()
r2.aggregateResults()

In [ ]:
datay = r1.USyearly
datac = r1.UScum


In [ ]:
datay_CdTe = r2.USyearly
datac_CdTe = r2.UScum

In [ ]:
filter_colc = [col for col in datay if col.startswith('WasteEOL')]
datay[filter_colc].to_csv(f'PVICE_RELOG_PCA_cSi_WasteEOL_{pv_ice_simulations}.csv')
filter_colc = [col for col in datay_CdTe if col.startswith('WasteEOL')]
datay_CdTe[filter_colc].to_csv(f'PVICE_RELOG_PCA_CdTe_WasteEOL_{pv_ice_simulations}.csv')